In [1]:
from __future__ import print_function
from keras.layers import Input, Dense, Activation, Dropout, LeakyReLU, Lambda, PReLU
from keras.models import Model, Sequential, model_from_json
from keras.callbacks import EarlyStopping
from keras import optimizers, activations, regularizers
from keras import backend as K
from sklearn.model_selection import train_test_split, KFold
from hyperopt import Trials, STATUS_OK, tpe
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py


Using TensorFlow backend.


In [7]:
'''Import data'''

# File path
data_orig = h5py.File('/Users/maggiezhang/Dropbox/AtmosphereMachineLearning/DensityData/JB2008_1999_2010_ROM_r100.mat', 'r')

# Reduced-order density data
dens_data = data_orig.get('densityDataLogVarROM100')
dens_data = np.array(dens_data)

# Space weather data
sw_data = data_orig.get('SWdataFull')
sw_data = np.array(sw_data)
sw_data = sw_data.transpose()
sw_col_names = ["doy", "UThrs", "F10", "F10B", "S10", "S10B", "XM10", "XM10B", "Y10", "Y10B", "DSTDTC", "GWRAS", "SUN(1)", "SUN(2)"]
df = pd.DataFrame(sw_data, columns = sw_col_names)
print(df.head())

# Only consider first 10 modes/dimensions of reduced data
dens_data = dens_data[:,0:10]


'''Training data'''
# Training input

# Testing the 70, 15, 15 for train, val, test 

split_15 = int(dens_data.shape[0]*0.85) # manual test split 

dens_data_train = dens_data[0:split_15,:]
sw_data_train = sw_data[0:split_15,:]
sw_data_train_append = sw_data[1:split_15+1,:] #looks at next sw timestep because it could be useful in prediction 
sw_data_train = np.concatenate((sw_data_train, sw_data_train_append), axis=1)
# Input is reduced-order density and space weather data
x_train = np.concatenate((dens_data_train, sw_data_train), axis=1)

# TODO: we could also add the SW data of the next time step if that perform better

# Training output
# Output is reduced-order density data at next time step
y_train = dens_data[1:split_15+1,:]

'''ROM LINEAR MATRICES''' 
Om = np.append(np.transpose(dens_data_train), np.transpose(sw_data_train), axis = 0)
X2 = np.transpose(y_train)

# Compute dynamic matrices:
# Phi = X2*pinv(Om)
Phi = np.matmul(X2,np.linalg.pinv(Om))

r = 10 #reduced order 
A = Phi[:r, :r]
B = Phi[:r, r:]

'''Test data'''
# Test input
dens_data_test = dens_data[split_15:-1,:]
sw_data_test = sw_data[split_15:-1,:]
sw_data_test_append = sw_data[split_15+1:,:]
sw_data_test = np.concatenate((sw_data_test, sw_data_test_append), axis = 1)
# Input is reduced-order density and space weather data
x_test = np.concatenate((dens_data_test, sw_data_test), axis=1)

# Test output
# Output is reduced-order density data at next time step
y_test = dens_data[split_15+1:,:]


    doy  UThrs    F10   F10B    S10   S10B   XM10  XM10B    Y10   Y10B  \
0  75.0   12.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
1  75.0   13.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
2  75.0   14.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
3  75.0   15.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
4  75.0   16.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   

   DSTDTC     GWRAS    SUN(1)    SUN(2)  
0    17.5  6.172181 -0.072585 -0.031433  
1    38.5  0.151512 -0.071919 -0.031146  
2    38.5  0.414028 -0.071254 -0.030859  
3    38.5  0.676544 -0.070589 -0.030571  
4    38.5  0.939060 -0.069923 -0.030284  


In [10]:
def swish(x, beta=1.0):
    return x * K.sigmoid(beta * x)

def data():
    data_orig = h5py.File('/Users/maggiezhang/Dropbox/AtmosphereMachineLearning/DensityData/JB2008_1999_2010_ROM_r100.mat', 'r')

    dens_data = data_orig.get('densityDataLogVarROM100')
    dens_data = np.array(dens_data)

    sw_data = data_orig.get('SWdataFull')
    sw_data = np.array(sw_data)
    sw_data = sw_data.transpose()

    dens_data = dens_data[:,0:10]
    dens_data_train = dens_data[0:-1,:]
    sw_data_train = sw_data[0:-1,:]
    sw_data_train_append = sw_data[1:,:]
    sw_data_train = np.concatenate((sw_data_train, sw_data_train_append), axis=1)
    x_train = np.concatenate((dens_data_train, sw_data_train), axis=1)
    y_train = dens_data[1:,:]

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15)
    
    return x_train, y_train, x_test, y_test

def create_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(Dense({{choice([120, 80, 40])}}, kernel_initializer='random_uniform'))
    model.add({{choice([Activation(lambda x: activations.relu(x, max_value = 100, alpha=0.3)), Activation(lambda x: activations.relu(x, max_value = 10, alpha=0.3)),Activation(lambda x: activations.relu(x, max_value = 6, alpha=0.3))])}})
    model.add(Dense(40, kernel_initializer='random_uniform'))
    model.add({{choice([Activation(lambda x: activations.relu(x, max_value = 100, alpha=0.3)), Activation(lambda x: activations.relu(x, max_value = 10, alpha=0.3)), Activation(lambda x: activations.relu(x, max_value = 6, alpha=0.3))])}})
    model.add(Dense(20, kernel_initializer='random_uniform'))
    model.add({{choice([Activation(lambda x: activations.relu(x, max_value = 100, alpha=0.3)), Activation(lambda x: activations.relu(x, max_value = 10, alpha=0.3)), Activation(lambda x: activations.relu(x, max_value = 6, alpha=0.3))])}})
    model.add(Dense(10, kernel_initializer='random_uniform'))
    
    model.compile(optimizer=optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False), loss='mse', metrics=['accuracy'])
    x_train_1, x_val, y_train_1, y_val = train_test_split(x_train, y_train, test_size=0.17647)
    result = model.fit(x = x_train_1, y = y_train_1, validation_data = (x_val, y_val), epochs = 100, batch_size = {{choice([16, 32])}}, verbose=2) 

    validation_loss = np.amin(result.history['val_loss']) 
    print('Best validation loss of epoch:', validation_loss)
    
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

best_run, best_model = optim.minimize(model=create_model,data=data,algo=tpe.suggest,max_evals=20,trials=Trials(),notebook_name='Hyperas_Tuning')
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from keras.layers import Input, Dense, Activation, Dropout, LeakyReLU, Lambda, PReLU
except:
    pass

try:
    from keras.models import Model, Sequential, model_from_json
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from keras import optimizers, activations, regularizers
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from sklearn.model_selection import train_test_split, KFold
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import scipy.io
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import pandas as pd

 - 5s - loss: 0.0310 - accuracy: 0.9945 - val_loss: 0.0213 - val_accuracy: 0.9952

Epoch 32/100                                          
 - 4s - loss: 0.0304 - accuracy: 0.9942 - val_loss: 0.0429 - val_accuracy: 0.9953

Epoch 33/100                                          
 - 4s - loss: 0.0275 - accuracy: 0.9947 - val_loss: 0.0168 - val_accuracy: 0.9951

Epoch 34/100                                          
 - 4s - loss: 0.0275 - accuracy: 0.9947 - val_loss: 0.0304 - val_accuracy: 0.9945

Epoch 35/100                                          
 - 5s - loss: 0.0275 - accuracy: 0.9950 - val_loss: 0.0259 - val_accuracy: 0.9957

Epoch 36/100                                          
 - 4s - loss: 0.0262 - accuracy: 0.9949 - val_loss: 0.0237 - val_accuracy: 0.9961

Epoch 37/100                                          
 - 4s - loss: 0.0255 - accuracy: 0.9955 - val_loss: 0.0288 - val_accuracy: 0.9936

Epoch 38/100                                          
 - 4s - loss: 0.0250 - accuracy: 0

 - 6s - loss: 0.0136 - accuracy: 0.9961 - val_loss: 0.0211 - val_accuracy: 0.9902

Epoch 91/100                                          
 - 5s - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.0078 - val_accuracy: 0.9963

Epoch 92/100                                          
 - 5s - loss: 0.0122 - accuracy: 0.9962 - val_loss: 0.0087 - val_accuracy: 0.9970

Epoch 93/100                                          
 - 5s - loss: 0.0123 - accuracy: 0.9964 - val_loss: 0.0136 - val_accuracy: 0.9926

Epoch 94/100                                          
 - 5s - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.0058 - val_accuracy: 0.9970

Epoch 95/100                                          
 - 5s - loss: 0.0126 - accuracy: 0.9965 - val_loss: 0.0057 - val_accuracy: 0.9968

Epoch 96/100                                          
 - 5s - loss: 0.0121 - accuracy: 0.9963 - val_loss: 0.0081 - val_accuracy: 0.9965

Epoch 97/100                                          
 - 6s - loss: 0.0120 - accuracy: 0

 - 5s - loss: 0.0313 - accuracy: 0.9932 - val_loss: 0.0576 - val_accuracy: 0.9917    

Epoch 39/100                                                                         
 - 5s - loss: 0.0305 - accuracy: 0.9935 - val_loss: 0.0233 - val_accuracy: 0.9941    

Epoch 40/100                                                                         
 - 6s - loss: 0.0307 - accuracy: 0.9938 - val_loss: 0.0526 - val_accuracy: 0.9842    

Epoch 41/100                                                                         
 - 6s - loss: 0.0296 - accuracy: 0.9933 - val_loss: 0.0245 - val_accuracy: 0.9937    

Epoch 42/100                                                                         
 - 6s - loss: 0.0289 - accuracy: 0.9939 - val_loss: 0.0222 - val_accuracy: 0.9934    

Epoch 43/100                                                                         
 - 7s - loss: 0.0281 - accuracy: 0.9939 - val_loss: 0.0388 - val_accuracy: 0.9936    

Epoch 44/100                                    

 - 6s - loss: 0.0172 - accuracy: 0.9956 - val_loss: 0.0154 - val_accuracy: 0.9953    

Epoch 86/100                                                                         
 - 6s - loss: 0.0166 - accuracy: 0.9959 - val_loss: 0.0183 - val_accuracy: 0.9925    

Epoch 87/100                                                                         
 - 6s - loss: 0.0174 - accuracy: 0.9960 - val_loss: 0.0176 - val_accuracy: 0.9948    

Epoch 88/100                                                                         
 - 6s - loss: 0.0165 - accuracy: 0.9955 - val_loss: 0.0117 - val_accuracy: 0.9954    

Epoch 89/100                                                                         
 - 5s - loss: 0.0164 - accuracy: 0.9957 - val_loss: 0.0159 - val_accuracy: 0.9958    

Epoch 90/100                                                                         
 - 5s - loss: 0.0167 - accuracy: 0.9958 - val_loss: 0.0118 - val_accuracy: 0.9965    

Epoch 91/100                                    

...

 - 2s - loss: 0.0160 - accuracy: 0.9959 - val_loss: 0.0082 - val_accuracy: 0.9976     

Epoch 96/100                                                                          
 - 2s - loss: 0.0141 - accuracy: 0.9961 - val_loss: 0.0130 - val_accuracy: 0.9966     

Epoch 97/100                                                                          
 - 2s - loss: 0.0149 - accuracy: 0.9961 - val_loss: 0.0095 - val_accuracy: 0.9968     

Epoch 98/100                                                                          
 - 2s - loss: 0.0146 - accuracy: 0.9961 - val_loss: 0.0129 - val_accuracy: 0.9973     

Epoch 99/100                                                                          
 - 2s - loss: 0.0147 - accuracy: 0.9961 - val_loss: 0.0224 - val_accuracy: 0.9940     

Epoch 100/100                                                                         
 - 2s - loss: 0.0138 - accuracy: 0.9963 - val_loss: 0.0134 - val_accuracy: 0.9971     

Best validation loss of epoch:       

 - 4s - loss: 0.0283 - accuracy: 0.9944 - val_loss: 0.0274 - val_accuracy: 0.9892     

Epoch 41/100                                                                          
 - 4s - loss: 0.0275 - accuracy: 0.9944 - val_loss: 0.0529 - val_accuracy: 0.9943     

Epoch 42/100                                                                          
 - 4s - loss: 0.0273 - accuracy: 0.9945 - val_loss: 0.0491 - val_accuracy: 0.9947     

Epoch 43/100                                                                          
 - 4s - loss: 0.0266 - accuracy: 0.9947 - val_loss: 0.0188 - val_accuracy: 0.9958     

Epoch 44/100                                                                          
 - 4s - loss: 0.0260 - accuracy: 0.9943 - val_loss: 0.0237 - val_accuracy: 0.9934     

Epoch 45/100                                                                          
 - 4s - loss: 0.0255 - accuracy: 0.9949 - val_loss: 0.0228 - val_accuracy: 0.9953     

Epoch 46/100                         

Epoch 87/100                                                                          
 - 4s - loss: 0.0161 - accuracy: 0.9961 - val_loss: 0.0155 - val_accuracy: 0.9976     

Epoch 88/100                                                                          
 - 4s - loss: 0.0166 - accuracy: 0.9961 - val_loss: 0.0117 - val_accuracy: 0.9969     

Epoch 89/100                                                                          
 - 4s - loss: 0.0158 - accuracy: 0.9960 - val_loss: 0.0131 - val_accuracy: 0.9970     

Epoch 90/100                                                                          
 - 4s - loss: 0.0156 - accuracy: 0.9959 - val_loss: 0.0210 - val_accuracy: 0.9963     

Epoch 91/100                                                                          
 - 4s - loss: 0.0157 - accuracy: 0.9960 - val_loss: 0.0196 - val_accuracy: 0.9972     

Epoch 92/100                                                                          
 - 4s - loss: 0.0157 - accuracy: 0.995

Epoch 32/100                                                                          
 - 2s - loss: 0.0337 - accuracy: 0.9932 - val_loss: 0.0439 - val_accuracy: 0.9934     

Epoch 33/100                                                                          
 - 2s - loss: 0.0328 - accuracy: 0.9933 - val_loss: 0.0267 - val_accuracy: 0.9948     

Epoch 34/100                                                                          
 - 2s - loss: 0.0325 - accuracy: 0.9936 - val_loss: 0.0274 - val_accuracy: 0.9947     

Epoch 35/100                                                                          
 - 2s - loss: 0.0310 - accuracy: 0.9935 - val_loss: 0.0339 - val_accuracy: 0.9932     

Epoch 36/100                                                                          
 - 2s - loss: 0.0303 - accuracy: 0.9936 - val_loss: 0.0391 - val_accuracy: 0.9894     

Epoch 37/100                                                                          
 - 2s - loss: 0.0313 - accuracy: 0.993

 - 2s - loss: 0.0182 - accuracy: 0.9955 - val_loss: 0.0146 - val_accuracy: 0.9943     

Epoch 79/100                                                                          
 - 2s - loss: 0.0173 - accuracy: 0.9954 - val_loss: 0.0111 - val_accuracy: 0.9962     

Epoch 80/100                                                                          
 - 2s - loss: 0.0165 - accuracy: 0.9954 - val_loss: 0.0116 - val_accuracy: 0.9955     

Epoch 81/100                                                                          
 - 2s - loss: 0.0172 - accuracy: 0.9954 - val_loss: 0.0126 - val_accuracy: 0.9934     

Epoch 82/100                                                                          
 - 2s - loss: 0.0174 - accuracy: 0.9952 - val_loss: 0.0316 - val_accuracy: 0.9956     

Epoch 83/100                                                                          
 - 2s - loss: 0.0159 - accuracy: 0.9954 - val_loss: 0.0213 - val_accuracy: 0.9930     

Epoch 84/100                         